In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import astropy.io.fits as fits
from scipy.interpolate import interp1d
from scipy import interpolate
from scipy.interpolate import UnivariateSpline
import pandas as pd


In [ ]:
panstar_response=pd.read_table('/share/data/jiwei/Lejeune1997/ps1.bandpass', sep = '\s+')#\s* response curve

In [ ]:
 #卷积计算g,r,i,z 流量
    
    ind=panstar_response.index.values
    m_g=panstar_response['g'][30:236].values
    m_g_wave=panstar_response['wave'][30:236].values*10
    m_r=panstar_response['r'][151:378].values
    m_r_wave=panstar_response['wave'][151:378].values*10
    i_ind=ind[301:500]
    m_i=panstar_response['i'][i_ind].values
    m_i_wave=panstar_response['wave'][i_ind].values*10
    z_ind=ind[403:521]
    m_z=panstar_response['z'][z_ind].values
    m_z_wave=panstar_response['wave'][z_ind].values*10
    
    #--------------------------------------------------------------------
    #在response 每个波段范围内的流量分布
    
    g1 = spectra_cali_lm_ex[0:2051]
    r1=spectra_cali_lm_ex[1210:3471]
    i1=spectra_cali_lm_ex[2710:4691] 
    z1=spectra_cali_lm_ex[3730:4901]
   
    #--------------------------------------------------------------------
    #因为response curve 与光谱波长不一致，所以进行插值到一个scale
    
    f_res_g=interpolate.interp1d(m_g_wave,m_g,kind='linear')#Flam
    fnew_res_g=f_res_g(wv1[0:2051])
    f_res_r=interpolate.interp1d(m_r_wave,m_r,kind='linear')#Flam
    fnew_res_r=f_res_r(wv1[1210:3471])
    f_res_i=interpolate.interp1d(m_i_wave,m_i,kind='linear')#Flam
    fnew_res_i=f_res_i(wv1[2710:4691])
    f_res_z=interpolate.interp1d(m_z_wave,m_z,kind='linear')#Flam
    fnew_res_z=f_res_z(wv1[3730:4901])
    print(fnew_res_i*i0,'fnew_res_i*i1')

    #卷积到流量
    g_mean=np.sum(fnew_res_g*g1)/np.sum(fnew_res_g)
    r_mean=np.sum(fnew_res_r*r1)/np.sum(fnew_res_r)
    i_mean=np.sum(fnew_res_i*i0)/np.sum(fnew_res_i)
    z_mean=np.sum(fnew_res_z*z1)/np.sum(fnew_res_z)
    
    #-----------------------------------------------------------------------
    
    #从流量计算星等
    
    X1 = np.array([0.481,0.617,0.752])#g,r,i,z中心波长
    SED_flux = np.array([g_mean,r_mean,i_mean,z_mean])
    
    #由于我的流量单位是erg/s/cm2/$\AA$，所以可以利用第一条代码将流量单位转换到jansky,利用第二条代码 将流量计算到AB星等
    AB_mag_jy = SED_fiux/((2.997925*(10**18))/((X1*10000)**2))/10**(-23)
    AB_mag = -2.5*(np.log10(AB_mag_jy)-23)-48.6